# Short story of change

As with all major systems in technology (like browsers) there is a constant state of "platform wars", where the different solutions and ecosystems vie for dominance. This is also the case with deep learning frameworks.

During it's course of development, Tensorflow amassed a seizable community, just by the virtue of being a large scale, deployable system with a giant as Google backing it. None the less, developing TF 1.x based code was absolutely not a smooth experience. 

The concept of execution Sessions and "define-and-run" made debugging clumsy and complicated, the programming solutions used were unlike other "standard" systems, and the whole thing was just somehow "un-pythonic".

This also paved the way for the emergence of new competitors, most notably [Pytorch](https://pytorch.org/) backed by Facebook Research (and very much inspired by Chainer). It is a full "pythonic" framework, with native conditional statements, command flow control, iteration, etc. It is a great tool for experimental code for new architectures, thus it became hugely popular in the research community. It was originally not an industry grade deployable framework, but it is being enhanced in that direction. 

None the less the popularity of Pytorch amongst the research community proved to be a major challenge for Tensorflow. In fact, in 2019 [this](https://thegradient.pub/state-of-ml-frameworks-2019-pytorch-dominates-research-tensorflow-dominates-industry/) article argued, that Pytorch is winning the game, since it has the backing of the research community. (Which is arguable...)

<mg src="https://thegradient.pub/content/images/2019/10/number_medium.png" width=45%>

Thus, with the 2.0 release of Tensorflow, Google announced a major overhaul, to become competitive again.



# Main changes

- Usability:
    - tf.keras as the de-facto high level API in TF
    -"Eager execution" enabled by default
- Clarity:
    - Removal of duplicate functionality
    - Consistent API
    - Consistent API in the whole ecosystem
- Flexibility:
    - Keeping the low level API
    - Deep access to internal ops (`tf.raw_ops`)
    - Inheritable interfaces for layers (and everything else)

Video of the [TF 2.0 intro event](https://www.youtube.com/watch?v=k5c-vg4rjBw) details the motivation and main changes, but the main takeaway is:

- **tf.Keras became the default layer abstraction**
- **Eager execution** is the norm

# The TF 2.x ecosystem

What remained - or in fact became more powerful - is the "production" ecosystem around TF.

<img src="http://drive.google.com/uc?export=view&id=18sayJe87HgVX1UpLeLDucX6TsWQoI9xt" width=65%>

From mobile and edge devices to hugely scalable server side deployments, there is a tool for everything.

Especially remarkable is [Tensorflow Serving](https://www.tensorflow.org/tfx/guide/serving), which enables huge scale deployment.

([This video](https://www.youtube.com/watch?v=q_IkJcPyNl0) is a nice intro to Serving.) 

# Developing models with TF 2.x

The "intended usage" of TF also changed a lot. According to the official sources, this is the suggested way / levels of development:

<img src="http://drive.google.com/uc?export=view&id=1pbSvYLtwO_8Q31S7kXk1KDbflSIanrYl" width=65%>

As we can see, tf.Keras is dominating the field, nearly no mention about "classic" TF ops and ways (which is now called "Tensorflow Core"). We also agree, that for most use cases, tf.Keras is the way to go - so maybe __learning the functional API__ is the most efficient use of one's time.

A very nice intro to the layers of development with TF 2.x can be found [here](https://www.youtube.com/watch?v=5ECD8J3dvDQ). We will follow it with our discussion below.

None the less, let's revisit, how TF "core" changed with Eager. 

# Eager execution

In [ ]:
import tensorflow as tf

tf.__version__

'2.0.0'

The dominating change: it is now **"define-by-run", so whatever we declare, it gets executed instantly!**

In [ ]:
a = tf.random.normal(shape=(2,3))
a

<tf.Tensor: id=69, shape=(2, 3), dtype=float32, numpy=
array([[ 0.64984685, -0.75238955, -0.13366635],
       [ 0.63856125, -0.6594663 ,  0.08644307]], dtype=float32)>

In [ ]:
b = tf.random.normal(shape=(2,3))
a+b

<tf.Tensor: id=76, shape=(2, 3), dtype=float32, numpy=
array([[ 1.694922  ,  0.37309778, -1.7960442 ],
       [ 2.3307428 , -1.2173747 , -0.6468137 ]], dtype=float32)>

Inside TF, there is a normal Numpy array, that is neatly accessible, we can manipulate it freely!

In [ ]:
b.numpy()

array([[ 1.0450752 ,  1.1254873 , -1.6623778 ],
       [ 1.6921815 , -0.55790836, -0.73325676]], dtype=float32)

In [ ]:
c = tf.Variable([1,2,3])
print(c)
c.assign([4,5,6]) 
c

<tf.Variable 'Variable:0' shape=(3,) dtype=int32, numpy=array([1, 2, 3], dtype=int32)>


<tf.Variable 'Variable:0' shape=(3,) dtype=int32, numpy=array([4, 5, 6], dtype=int32)>

(Caveat: a slight distinction of general Tensors and Variables still lingers, only Variables can be assigned.)

## Gradient tape

Now - akin to Pytorch - the "gradient tape" is a strong abstraction in low level TF 2.x. It allows for the collection of gradient information, and it's application to dynamically defined variables.

In [ ]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y=x*x
dy_dx = g.gradient(y,x)

dy_dx

<tf.Tensor: id=92, shape=(), dtype=float32, numpy=6.0>

Please observe, that variables in a tape scope get automatically tracked.

In [ ]:
dense1 = tf.keras.layers.Dense(32)
dense2 = tf.keras.layers.Dense(32)

with tf.GradientTape() as tape:
    result = dense2(dense1(tf.zeros([1,10])))
    grad = tape.gradient(result, dense1.variables)

grad[1]

<tf.Tensor: id=678, shape=(32,), dtype=float32, numpy=
array([ 0.2144123 ,  0.9980367 ,  2.523283  , -0.05389541,  1.2221451 ,
        1.6887089 ,  0.38739684,  2.1451378 , -0.7038405 ,  2.1398394 ,
       -0.33315706,  0.65045196, -0.25495082, -2.2888103 ,  0.6497345 ,
        1.2448461 , -0.62869006,  0.13058892, -0.4034389 ,  0.53868127,
       -0.07362868,  1.2114792 , -0.28120014, -1.1203902 ,  0.3054561 ,
       -2.00901   , -0.591593  , -0.9884967 ,  0.71254337,  1.5557951 ,
       -0.969301  ,  0.09261519], dtype=float32)>

This facility is enabling the rapid development and automatic differentiation of complex architectures.

For a simple linear regression model pretty close to TF1, but already in TF2, see [here](https://towardsdatascience.com/get-started-with-tensorflow-2-0-and-linear-regression-29b5dbd65977) or for a different style [here](https://heartbeat.fritz.ai/linear-regression-using-tensorflow-2-0-1cd51e211e1f).

If you want to know, what happens under the hood of TF 2.x, eg. for the handling of conditionals, see [here](https://www.youtube.com/watch?v=IzKXEbpT9Lg)

# Speeding up custom functions

<img src="http://drive.google.com/uc?export=view&id=1IROhjuSRSzfEPMLWXmNUc_QiH70hTV5z" width=65%>

[Tensorflow @function decorator](https://www.tensorflow.org/api_docs/python/tf/function) is another nice facility. Even if we omit it, we can - as we saw above - write Pythonic code that gets executed eagerly, so we can build up complex structures in a functional manner. None the less, if we would like to make the code __run more efficiently__, we can use  the decorator, that results in TF trying to __compile our ops to a static graph, thus speeding it up__.